# Designing sound control systems

In [1]:
%pylab
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In the previous notebook we explored the time-harmonic wavefield with $P(x,y) = e^{-ikr}/r$ where $r$ is distance from the origin. In this notebook we'll be examining the effect superimposing two 'point sources' like this, one of which we'll call the primary source and one the secondary. We will therefore have:
$$ P(\mathbf{x}) = A_p \frac{e^{-ikr_p}}{r_p} + A_s\frac{e^{-ikr_s}}{r_s}.$$

$A_p$ and $A_s$ are the complex amlitudes of the two sources and $r_p$ and $r_s$  are distances from the sources which are located at $\mathbf{x}_p = (x_p, y_p, 0)$ and $\mathbf{x}_s = (x_s, y_s, 0)$.   We can animate this interference field but first we'll define a new function to save some typing:

In [2]:
def circspace(N):
    return linspace(0, 2*pi*(1 - 1/N),N)

We'll use tuples to store the $x$ and $y$ co-ordinates of $\mathbf{x}_p$ and $\mathbf{x}_s$.

In [47]:
x, y = meshgrid(linspace(-5, 5, 150), linspace(-5, 5, 150))
xp = (0, 5) # Primary source
xs = (5, 0) # Secondary source
As = 1;
As = 1;
rs = sqrt((x - xs[0])**2 + (y - xs[1])**2)
rp = sqrt((x - xp[0])**2 + (y - xp[1])**2)
fig = figure()
Nframes = 50
ims = []
k = 2*pi/1.5
P = Ap*exp(-1j*k*rp)/rp + As*exp(-1j*k*rs)/rs

for wt in circspace(Nframes):
    p = real(exp(1j*wt)*P)
    ims.append([pcolormesh(x, y, p, cmap=cm.coolwarm)])
    clim(-1,1)
    axis('equal')
    axis('off')

ani = animation.ArtistAnimation(fig, ims, interval=100)
show()

### Exercises
1. Explore the effect of changing the real and imaginary parts of $A_s$.
2. Explore the effect of moving the sources closer and further apart.

## Active control
Suppose we have a microphone at a location $\mathbf{x}_m$ and we want to adjust $A_s$ to minimise the sound it detects. In particular we want to  minimise $W_m = |P(\mathbf{x}_m)|^2$ which is proportional to the sound power at the microphone. Because it's proportional minimizing it will minimize the true sound power. We'll (incorrectly) refer to $W_m$ as the sound power from now on - you'll learn the exact expression in your Acoustics lectures. We can explore how this quantity varies with $\Re\{A_s\}$ as $\Im\{A_s\}$ like this:

In [48]:
xp = (0, 5)                     # Primary source
xs = (5, 0)                     # Secondary source
xm = (0, 0)                     # Microphone position
rsm = sqrt((xm[0] - xs[0])**2 + (xm[1] - xs[1])**2)
rpm = sqrt((xm[0] - xp[0])**2 + (xm[1] - xp[1])**2)
N = 50                          # Number of steps
Ap = 1
As_space = linspace(-2, 2, N)
Asr, Asi = meshgrid(As_space, As_space)
Wm = zeros_like(Asr)

for i in range(len(As_space)):
    for j in range(len(As_space)):
        Pm = Ap*exp(-1j*k*rpm)/rpm + (Asr[i, j] +1j*Asi[i, j])*exp(-1j*k*rsm)/rsm
        Wm[i, j] = absolute(Pm)**2

fig = figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(Asr, Asi, Wm, rstride=1, cstride=1, cmap=cm.cool)
xlabel('Real')
ylabel('Imaginary')
axis('equal')

#contour(Asr, Asi, Wm)


(-2.0, 2.0, -2.0, 2.0)

If we make a function that returns $W_m$ when given an array containing the real and imaginary parts of $A_s$ and some other arguments:

In [41]:
def power(As, Ap, xs, xp, xm):
    rsm = sqrt((xm[0] - xs[0])**2 + (xm[1] - xs[1])**2)
    rpm = sqrt((xm[0] - xp[0])**2 + (xm[1] - xp[1])**2)
    Pm = Ap*exp(-1j*k*rpm)/rpm + (As[0] +1j*As[1])*exp(-1j*k*rsm)/rsm
    return absolute(Pm)**2


...and import a [function that minimizes other functions:](http://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.optimize.fmin.html)

In [33]:
from scipy.optimize import fmin

...we can use it to find the value for $A_s$ that mimimizes the sound power at $\mathbf{x}_m$.

In [44]:
fmin(power, [0, 0], args = (1.0, (5, 0), (0, 5), (0, 0)))

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 61
         Function evaluations: 119


array([ -9.99962635e-01,   3.76140017e-05])

### Final *assessed* exercise
In this exercise you will use a time-harmonic point source with complex amplitude $A_s$ to control the sound power at a point in a plane wave. Create a new notebook containing all thte explanation and code necessary to see what you've done. Your note book should include  the following:

1. Code that generates an animated wavefield with a plane wave and he field of a point source superimposed.

2. Code that generates a surface plot that shows how $W_m$ varies with $\Re\{A_s\}$ and $\Im\{A_s\}$ for this problem.

3. Code that uses `fmin()` to find the value of $A_s$ that minimizes sound power at a point $\mathbf{x}_m$ in the field.

4. Code that generates another animation that uses the value of $A_s$ that you found in the previous step.

5. Enough mark-up cells that someone could read the whole notebok and understand what you're doing.

You can choose the parameters of the primary plane wavefield as long as it isn't silent and you can choose the location to control at $\mathbf{x}_m$.

Email me your notebook by the end of the month.